In [66]:
!pip3 install sql-metadata

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
You should consider upgrading via the '/usr/local/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.


In [41]:
!pip3 install sqlparse

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
     |████████████████████████████████| 42 kB 2.0 MB/s eta 0:00:01
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
You should consider upgrading via the '/usr/local/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.


In [229]:
import pandas as pd
import sqlparse
import re
from collections import defaultdict
from sql_metadata import Parser
import pprint

# Parse logs using sql parser; get tables and columns being used 

In [272]:
def get_column_usage_from_logs(log_file_dataframe):

    valid_log_types = { "SELECT": True, "UPDATE": True }
    execute_unnamed_start_token = "execute <unnamed>:"
    execute_unnamed_start_token_len = len(execute_unnamed_start_token)
    transaction_regex = r"(execute S_[0-9]+: )(.*)"

    # Maintain parsing counters
    parsing_success = { 
        "success": 0,
        "success_log_type": defaultdict(lambda : 0),
        "failure": 0,
        "skipped": 0,
        "skipped_log_type": defaultdict(lambda : 0)
    }

    column_usage = defaultdict(lambda : defaultdict(lambda : 0))

    print ("Total log lines: ", len(log_file_dataframe))

    # Parse each log line
    log_count = 0
    for log_type, log in log_file_dataframe[[7, 13]].values:
        log_count += 1

        if log_count % 10000 == 0:
            print ("Processed log_count: ", log_count)
            pprint.pprint(parsing_success)
            print ("\n\n")

        # Ignore queries from log that are [ BEGIN, COMMIT ]
        if log_type not in valid_log_types:
            # Update counters
            parsing_success["skipped"] += 1
            parsing_success["skipped_log_type"][log_type] += 1
            continue

        if log.startswith(execute_unnamed_start_token):
            # Log line starts with "execute <unnamed>:"
            query = log[execute_unnamed_start_token_len:]
        else:
            # Log line starts with "execute S_<>"
            query = re.sub(transaction_regex, r"\2", log)

        try:
            # Update counters
            parsing_success["success"] += 1
            parsing_success["success_log_type"][log_type] += 1

            parsed_query = Parser(query)
            columns = parsed_query.columns_dict

            # Ignore pg system queries (touches any table that starts with "pq_")
            if any(map(lambda x : x.startswith("pg_"), parsed_query.tables)):
                continue

            # If only one table, prefix is not there; append
            table_prefix = ""
            if len(parsed_query.tables) == 1:
                table_prefix = parsed_query.tables[0] + "."

            for operator_type in columns:
                for column in columns[operator_type]:
                    if "." in column:
                        column_usage[operator_type][column] += 1
                    else:
#                         if table_prefix == "":
#                             print (columns, parsed_query.tables)
                        column_usage[operator_type][table_prefix + column] += 1
        except Exception as e:
            # Update counters 
            parsing_success["failure"] += 1
        

    return parsing_success, column_usage

# Load data from log csv

In [273]:
log_file_path = "data/epinion_scale_10.csv"
# log_file_path = "data/epinions_scale_100_rate_1000_time_60.csv"
log_file_dataframe = pd.read_csv(log_file_path, header=None)

# Process data; get column usage across operators

In [274]:
parsing_success, column_usage = get_column_usage_from_logs(log_file_dataframe)

Total log lines:  1920


In [275]:
pprint.pprint(parsing_success)

{'failure': 7,
 'skipped': 1237,
 'skipped_log_type': defaultdict(<function get_column_usage_from_logs.<locals>.<lambda> at 0x123191d30>,
                                 {nan: 11,
                                  'BEGIN': 605,
                                  'COMMIT': 605,
                                  'SET': 9,
                                  'SHOW': 1,
                                  'idle': 6}),
 'success': 683,
 'success_log_type': defaultdict(<function get_column_usage_from_logs.<locals>.<lambda> at 0x123191820>,
                                 {'SELECT': 383,
                                  'UPDATE': 300})}


In [276]:
for operator, usage in column_usage.items():
    print ("operator", operator)
    print (sorted(usage.items(), key = lambda x : -x[1]))
    print ("\n")

operator select
[('*', 127), ('review.rating', 62), ('rating', 59), ('review.*', 53), ('trust.*', 53), ('useracct.u_id', 1), ('item.i_id', 1)]


operator update
[('trust.trust', 140), ('review.rating', 56), ('item.title', 54), ('useracct.name', 50)]


operator where
[('review.i_id', 300), ('trust.source_u_id', 252), ('trust.target_u_id', 199), ('review.u_id', 172), ('item.i_id', 124), ('useracct.u_id', 107)]


operator order_by
[('review.creation_date', 180), ('rating', 127)]




# TODOs:

- Some columns do not have table prefixed (rating vs review.rating)